# Exam 2021, 8.00-13.00 for the course 1MS041 (Introduction to Data Science / Introduktion till dataanalys)

## Instructions:
1. Complete the problems by following instructions.
2. When done, submit this file with your solutions saved, following the instruction sheet.

This exam has 3 problems for a total of 40 points, to pass you need
20 points.

## Some general hints and information:
* Try to answer all questions even if you are uncertain.
* Comment your code, so that if you get the wrong answer I can understand how you thought
this can give you some points even though the code does not run.
* Follow the instruction sheet rigorously.
* This exam is partially autograded, but your code and your free text answers are manually graded anonymously.
* If there are any questions, please ask the exam guards, they will escalate it to me if necessary.
* I (Benny) will visit the exam room at around 10:30 to see if there are any questions.

## Tips for free text answers
* Be VERY clear with your reasoning, there should be zero ambiguity in what you are referring to.
* If you want to include math, you can write LaTeX in the Markdown cells, for instance `$f(x)=x^2$` will be rendered as $f(x)=x^2$ and `$$f(x) = x^2$$` will become an equation line, as follows
$$f(x) = x^2$$
Another example is `$$f_{Y \mid X}(y,x) = P(Y = y \mid X = x) = \exp(\alpha \cdot x + \beta)$$` which renders as
$$f_{Y \mid X}(y,x) = P(Y = y \mid X = x) = \exp(\alpha \cdot x + \beta)$$

## Finally some rules:
* You may not communicate with others during the exam, for example:
    * You cannot ask for help in Stack-Overflow or other such help forums during the Exam.
    * You may not communicate with AI's, for instance ChatGPT.
    * Your on-line and off-line activity is being monitored according to the examination rules.

## Good luck!

In [2]:
# Insert your anonymous exam ID as a string in the variable below
examID="XXX"


---
## Exam vB, PROBLEM 1
Maximum Points = 8


## Probability warmup
Let's say we have an exam question which consists of $20$ yes/no questions. 
From past performance of similar students, a randomly chosen student will know the correct answer to $N \sim \text{binom}(20,11/20)$ questions. Furthermore, we assume that the student will guess the answer with equal probability to each question they don't know the answer to, i.e. given $N$ we define $Z \sim \text{binom}(20-N,1/2)$ as the number of correctly guessed answers. Define $Y = N + Z$, i.e., $Y$ represents the number of total correct answers.

We are interested in setting a deterministic threshold $T$, i.e., we would pass a student at threshold $T$ if $Y \geq T$. Here $T \in \{0,1,2,\ldots,20\}$.

1. [5p] For each threshold $T$, compute the probability that the student *knows* less than $10$ correct answers given that the student passed, i.e., $N < 10$. Put the answer in `problem11_probabilities` as a list.
2. [3p] What is the smallest value of $T$ such that if $Y \geq T$ then we are 90\% certain that $N \geq 10$?

In [3]:
from scipy.special import binom as binomial
# Hint the PMF of N is p_N(k) where p_N is
p_n = 11/20
p_N = lambda k: binomial(20,k)*((1-p_n)**(20-k))*((p_n)**k)
p_z = 1/2
p_Z = lambda k, n: binomial(20-n,k)*((1-p_z)**(20-n-k))*((p_z)**k)

In [4]:
from itertools import product

probs = [0 for i in range(21)]
p_n_10 = sum([p_N(i) for i in range(10)])
#print(p_n_10)

for t in range(21):
    #print(t)
    combs_cond = [i for i in product(range(t), repeat=2) if i[0]+i[1] < t and i[0] < 10]
    p_cond = sum([p_N(N)*p_Z(Z,N) for N, Z in combs_cond])
    combs = [i for i in product(range(t), repeat=2) if i[0]+i[1] < t]
    #print(combs)
    p = sum([p_N(N)*p_Z(Z,N) for N, Z in combs])
    #print(p_cond, p, p_n_10)
    probs[t] = (p_n_10 - p_cond) / (1-p)
    

print(probs)

[0.24928935982841183, 0.2492893598283288, 0.24928935982261044, 0.24928935963549273, 0.24928935576839328, 0.24928929915834963, 0.24928867518930245, 0.24928330207958452, 0.2492462852336602, 0.24903902630299052, 0.24808569900431432, 0.2446082001497593, 0.23494396957815233, 0.2147564151317591, 0.18267139196620993, 0.1427252244707204, 0.10227042692681905, 0.06762809950564569, 0.04166472439122733, 0.02415113434042271, 0.01328746267959387]


In [5]:

# Part 1: 
# replace XXX to represent P(N < 10) for T = [0,1,2,...,20], i.e. your answer should be a list
# of length 21.
problem11_probabilities = probs

In [6]:

# Part 2: Give an integer between 0 and 10 which is the answer to 2.
def get_t(probs):
    for t, res in enumerate(probs):
        # print(t, 1-res)
        if (1-res) > 0.9:
            return t

get_t(problem11_probabilities)

17

In [7]:

# Part 2: Give an integer between 0 and 20 which is the answer to 2.
problem12_T = get_t(problem11_probabilities)

---
## Exam vB, PROBLEM 2
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [8]:

def problem2_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """

    m = 2**31
    a = 1103515245
    b = 12345

    x = seed # the seed
    retValue = [x % m]  # start the list with x=x0
    for i in range(2, size+1, 1):
        x = (a * x + b) % m # the generator, using modular arithmetic
        retValue.append(x) # append the new x to the list
    return retValue

In [9]:

def problem2_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem2_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    nums = generator(size, seed)
    lst = [x/(period-1) for x in nums]
    
    return lst

In [10]:

def problem2_accept_reject(uniformGenerator=None, size=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    import numpy as np
    
    samples = []
    x = uniformGenerator(size=size, seed=seed)
    y = uniformGenerator(size=size, seed=seed)
    for x,y in zip(x,y):
        if (y*(np.pi/2)) <= ((np.pi/2)*np.abs(np.sin(x*2*np.pi))):
            samples.append(x)
    
    return samples

---
#### Local Test for Exam vB, PROBLEM 2
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [11]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem2_LCG(size=10, seed = 1))

period = 2**31

print("Uniform sampler %s" % problem2_uniform(generator=problem2_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem2_uniform(generator=problem2_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem2_accept_reject(uniformGenerator = uniform_sampler,size=20,seed=1))

LCG output: [1, 1103527590, 377401575, 662824084, 1147902781, 2035015474, 368800899, 1508029952, 486256185, 1062517886]
Uniform sampler [4.656612875245797e-10, 0.5138700783782965, 0.17574130332830423, 0.3086515163577402, 0.5345338869535057, 0.9476279257552829, 0.17173630146856247, 0.7022311690739501, 0.2264306811738902, 0.49477344681265456]
Accept-Reject sampler [4.656612875245797e-10, 0.17574130332830423, 0.3086515163577402, 0.17173630146856247, 0.7022311690739501, 0.2264306811738902, 0.1247203196979688, 0.08389880325826761, 0.38964712125698436, 0.2772257971936957, 0.3680707194693716, 0.7656819032345349, 0.646473150535707, 0.7671388111855549]


In [12]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    set_random_seed(seed)
    
    return [random() for s in range(size)]

print("Accept-Reject sampler %s" % problem2_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

TypeError: problem2_accept_reject() got an unexpected keyword argument 'n_iterations'

---
## Exam vB, PROBLEM 3
Maximum Points = 8


## Concentration of measure

As you recall, we said that concentration of measure was simply the phenomenon where we expect that the probability of a large deviation of some quantity becoming smaller as we observe more samples: [0.4 points per correct answer]

1. Which of the following will exponentially concentrate, i.e. for some $C_1,C_2,C_3,C_4 $ 
$$
    P(Z - \mathbb{E}[Z] \geq \epsilon) \leq C_1 e^{-C_2 n \epsilon^2} \wedge C_3 e^{-C_4 n (\epsilon+1)} \enspace .
$$

    1. The empirical mean of i.i.d. sub-Gaussian random variables?
    2. The empirical mean of i.i.d. sub-Exponential random variables?
    3. The empirical mean of i.i.d. random variables with finite variance?
    4. The empirical variance of i.i.d. random variables with finite variance?
    5. The empirical variance of i.i.d. sub-Gaussian random variables?
    6. The empirical variance of i.i.d. sub-Exponential random variables?
    7. The empirical third moment of i.i.d. sub-Gaussian random variables?
    8. The empirical fourth moment of i.i.d. sub-Gaussian random variables?
    9. The empirical mean of i.i.d. deterministic random variables?
    10. The empirical tenth moment of i.i.d. Bernoulli random variables?

2. Which of the above will concentrate in the weaker sense, that for some $C_1$
$$
    P(Z - \mathbb{E}[Z] \geq \epsilon) \leq \frac{C_1}{n \epsilon^2}?
$$

In [ ]:

# Answers to part 1, which of the alternatives exponentially concentrate, answer as a list
# i.e. [1,4,5] that is example 1, 4, and 5 concentrate
problem3_answer_1 = [XXX]

In [ ]:

# Answers to part 2, which of the alternatives concentrate in the weaker sense, answer as a list
# i.e. [1,4,5] that is example 1, 4, and 5 concentrate
problem3_answer_2 = [XXX]

---
## Exam vB, PROBLEM 4
Maximum Points = 8


## SMS spam filtering [8p]

In the following problem we will explore SMS spam texts. The dataset is the `SMS Spam Collection Dataset` and we have provided for you a way to load the data. If you run the appropriate cell below, the result will be in the `spam_no_spam` variable. The result is a `list` of `tuples` with the first position in the tuple being the SMS text and the second being a flag `0 = not spam` and `1 = spam`.

1. [3p] Let $X$ be the random variable that represents each SMS text (an entry in the list), and let $Y$ represent whether text is spam or not i.e. $Y \in \{0,1\}$. Thus $\mathbb{P}(Y = 1)$ is the probability that we get a spam. The goal is to estimate:
$$
    \mathbb{P}(Y = 1 | \text{"free" or "prize" is in } X) \enspace .
$$
That is, the probability that the SMS is spam given that "free" or "prize" occurs in the SMS. 
Hint: it is good to remove the upper/lower case of words so that we can also find "Free" and "Prize"; this can be done with `text.lower()` if `text` a string.
2. [3p] Provide a "90\%" interval of confidence around the true probability. I.e. use the Hoeffding inequality to obtain for your estimate $\hat P$ of the above quantity. Find $l > 0$ such that the following holds:
$$
    \mathbb{P}(\hat P - l \leq \mathbb{E}[\hat P] \leq \hat P + l) \geq 0.9 \enspace .
$$
3. [2p] Repeat the two exercises above for "free" appearing twice in the SMS.

In [13]:

# Run this cell to get the SMS text data
from Utils import load_sms
spam_no_spam = load_sms()

In [14]:
spam_no_spam[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  0),
 ('Ok lar... Joking wif u oni...', 0),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  1)]

In [15]:
import re
check = lambda x: 1 if re.search("\\bfree\\b|\\bprize\\b", x, re.I) is not None else 0
# fill in the estimate for part 1 here (should be a number between 0 and 1)
spam_and_word = [1 if check(x[0])==1 and x[1]==1 else 0 for x in spam_no_spam]
word = [check(x[0]) for x in spam_no_spam]
print(sum(spam_and_word))
problem4_hatP = sum(spam_and_word)/sum(word)
print(problem4_hatP)

251
0.8096774193548387


In [16]:
import numpy as np
# fill in the calculated l from part 2 here
problem4_l = np.sqrt((-1/(2*sum(word)))*np.log(0.1/2))
print(problem4_l)

0.0695113389862958


In [17]:
len(re.findall("\\bfree\\b", spam_no_spam[2][0], re.I)) == 2

False

In [18]:
check_2 = lambda x: 1 if len(re.findall("\\bfree\\b", x, re.I)) == 2 else 0
# fill in the estimate for hatP for the double free question in part 3 here (should be a number between 0 and 1)
spam_and_word_2 = [1 if check_2(x[0])==1 and x[1]==1 else 0 for x in spam_no_spam]
word_2 = [check_2(x[0]) for x in spam_no_spam]
print(sum(spam_and_word_2))
problem4_hatP2 = sum(spam_and_word_2)/sum(word_2)
print(problem4_hatP2)

30
0.967741935483871


In [19]:

# fill in the estimate for l for the double free question in part 3 here
problem4_l2 = np.sqrt((-1/(2*sum(word_2)))*np.log(0.1/2))
print(problem4_l2)

0.21981415440475455


---
## Exam vB, PROBLEM 5
Maximum Points = 8


## Markovian travel

The dataset `Travel Dataset - Datathon 2019` is a simulated dataset designed to mimic real corporate travel systems -- focusing on flights and hotels. The file is at `data/flights.csv` in the same folder as `Exam.ipynb`, i.e. you can use the path `data/flights.csv` from the notebook to access the file.

1. [2p] In the first code-box 
    1. Load the csv from file `data/flights.csv`
    2. Fill in the value of the variables as specified by their names.
2. [2p] In the second code-box your goal is to estimate a Markov chain transition matrix for the travels of these users. For example, if we enumerate the cities according to alphabetical order, the first city `'Aracaju (SE)'` would correspond to $0$. Each row of the file corresponds to one flight, i.e. it has a starting city and an ending city. We model this as a stationary Markov chain, i.e. each user's travel trajectory is a realization of the Markov chain, $X_t$. Here, $X_t$ is the current city the user is at, at step $t$, and $X_{t+1}$ is the city the user travels to at the next time step. This means that to each row in the file there is a corresponding pair $(X_{t},X_{t+1})$. The stationarity assumption gives that for all $t$ there is a transition density $p$ such that $P(X_{t+1} = y | X_t = x) = p(x,y)$ (for all $x,y$). The transition matrix should be `n_cities` x `n_citites` in size.
3. [2p] Use the transition matrix to compute out the stationary distribution.
4. [2p] Given that we start in 'Aracaju (SE)' what is the probability that after 3 steps we will be back in 'Aracaju (SE)'?

In [1]:
import pandas as pd

data = pd.read_csv("../data/flights.csv")
#print(data.head())
number_of_cities = pd.concat([data["from"], data["to"]]).nunique()
print(number_of_cities)
number_of_userCodes = data["userCode"].nunique()
print(number_of_userCodes)
number_of_observations = data.shape[0]
print(number_of_observations)

9
1335
271888


In [2]:

# This is a very useful function that you can use for part 2. You have seen this before when parsing the
# pride and prejudice book.

def makeFreqDict(myDataList):
    '''Make a frequency mapping out of a list of data.

    Param myDataList, a list of data.
    Return a dictionary mapping each unique data value to its frequency count.'''

    freqDict = {} # start with an empty dictionary

    for res in myDataList:
        if res in freqDict: # the data value already exists as a key
                freqDict[res] = freqDict[res] + 1 # add 1 to the count using sage integers
        else: # the data value does not exist as a key value
            freqDict[res] = 1 # add a new key-value pair for this new data value, frequency 1

    return freqDict # return the dictionary created

In [43]:
from itertools import product
import numpy as np

cities = pd.concat([data["from"], data["to"]]).tolist() # A list of all cities
unique_cities = sorted(set(cities)) # The unique cities
n_cities = len(unique_cities) # The number of unique citites
print(n_cities)

# Count the different transitions
transitions = list(product(unique_cities, repeat=2)) # A list containing tuples ex: ('Aracaju (SE)','Rio de Janeiro (RJ)') of all transitions in the text
print(transitions)
transition_counts = makeFreqDict(list(zip(data["from"], data["to"]))) # A dictionary that counts the number of each transition 
# ex: ('Aracaju (SE)','Rio de Janeiro (RJ)'):4
print(transition_counts)
print(sum(transition_counts.values()))
indexToCity = {k: v for k, v in enumerate(unique_cities)} # A dictionary that maps the n-1 number to the n:th unique_city,
# ex: 0:'Aracaju (SE)'
print(indexToCity)
cityToIndex = {v: k for k, v in indexToCity.items()} # The inverse function of indexToWord, 
# ex: 'Aracaju (SE)':0
print(cityToIndex)

# Part 3, finding the maximum likelihood estimate of the transition matrix

transition_matrix = np.zeros((n_cities, n_cities)) # a numpy array of size (n_cities,n_cities)
for i in range(n_cities):
    divider = sum([transition_counts[(indexToCity[i], indexToCity[k])] if (indexToCity[i], indexToCity[k]) in transition_counts else 0 for k in range(n_cities)])
    for j in range(n_cities):
        transition_matrix[i,j] = transition_counts[(indexToCity[i], indexToCity[j])]/divider if (indexToCity[i], indexToCity[j]) in transition_counts else 0

print(transition_matrix)

# The transition matrix should be ordered in such a way that
# p_{'Aracaju (SE)','Rio de Janeiro (RJ)'} = transition_matrix[cityToIndex['Aracaju (SE)'],cityToIndex['Rio de Janeiro (RJ)']]
# and represents the probability of travelling Aracaju (SE)->Rio de Janeiro (RJ)

# Make sure that the transition_matrix does not contain np.nan from division by zero for instance

9
[('Aracaju (SE)', 'Aracaju (SE)'), ('Aracaju (SE)', 'Brasilia (DF)'), ('Aracaju (SE)', 'Campo Grande (MS)'), ('Aracaju (SE)', 'Florianopolis (SC)'), ('Aracaju (SE)', 'Natal (RN)'), ('Aracaju (SE)', 'Recife (PE)'), ('Aracaju (SE)', 'Rio de Janeiro (RJ)'), ('Aracaju (SE)', 'Salvador (BH)'), ('Aracaju (SE)', 'Sao Paulo (SP)'), ('Brasilia (DF)', 'Aracaju (SE)'), ('Brasilia (DF)', 'Brasilia (DF)'), ('Brasilia (DF)', 'Campo Grande (MS)'), ('Brasilia (DF)', 'Florianopolis (SC)'), ('Brasilia (DF)', 'Natal (RN)'), ('Brasilia (DF)', 'Recife (PE)'), ('Brasilia (DF)', 'Rio de Janeiro (RJ)'), ('Brasilia (DF)', 'Salvador (BH)'), ('Brasilia (DF)', 'Sao Paulo (SP)'), ('Campo Grande (MS)', 'Aracaju (SE)'), ('Campo Grande (MS)', 'Brasilia (DF)'), ('Campo Grande (MS)', 'Campo Grande (MS)'), ('Campo Grande (MS)', 'Florianopolis (SC)'), ('Campo Grande (MS)', 'Natal (RN)'), ('Campo Grande (MS)', 'Recife (PE)'), ('Campo Grande (MS)', 'Rio de Janeiro (RJ)'), ('Campo Grande (MS)', 'Salvador (BH)'), ('Campo G

In [44]:

# This should be a numpy array of length n_cities which sums to 1 and is all positive
eigenvalues, eigenvectors = np.linalg.eig(transition_matrix.T)
print(eigenvalues)

stationary_distribution_problem5 = eigenvectors[:,0]/sum(eigenvectors[:,0])
print(stationary_distribution_problem5)

[ 1.00000000e+00 -3.14924841e-01  1.11731432e-04 -5.29756651e-02
 -7.76523120e-02 -1.58563203e-01 -1.46602836e-01 -1.25937067e-01
 -1.23455807e-01]
[0.13690932 0.1132047  0.12780262 0.21081107 0.08752133 0.11210498
 0.06184532 0.06290826 0.0868924 ]


In [45]:
from itertools import repeat
# Compute the return probability for part 3 of problem 5
# print(unique_cities[0])
step1 = list(zip(repeat(unique_cities[0], times=n_cities), unique_cities))
print(len(step1))
step2 = list(product(step1, repeat=2))
print(len(step2))
temp_step3 = [i for i in zip(unique_cities, repeat(unique_cities[0], times=n_cities))]
print(len(temp_step3*n_cities))
step3 = [(i[0], i[1], j) for i, j in zip(step2, temp_step3*n_cities)]
print(len(step3))
#print(list(zip(step2, repeat(list(zip(unique_cities, repeat(unique_cities[0], times=n_cities))), times=n_cities))))
res = 0
for s1, s2, s3 in step3:
    # print(cityToIndex[c1], cityToIndex[c2])
    # print(f"s1: {s1}, s2: {s2}, s3: {s3}")
    res += transition_matrix[cityToIndex[s1[0]], cityToIndex[s1[1]]]*transition_matrix[cityToIndex[s2[0]], cityToIndex[s2[1]]]*transition_matrix[cityToIndex[s3[0]], cityToIndex[s3[1]]]

return_probability_problem5 = res
print(return_probability_problem5)

9
81
81
81
0.15887131217089695


---
#### Local Test for Exam vB, PROBLEM 5
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [46]:
# Once you have created all your functions, you can make a small test here to see
# what would be generated from your model.
import numpy as np

start = np.zeros(shape=(n_cities,1))
start[cityToIndex['Aracaju (SE)'],0] = 1

current_pos = start
for i in range(10):
    random_word_index = np.random.choice(range(n_cities),p=current_pos.reshape(-1))
    current_pos = np.zeros_like(start)
    current_pos[random_word_index] = 1
    print(indexToCity[random_word_index],end='->')
    current_pos = (current_pos.T@transition_matrix).T

Aracaju (SE)->Recife (PE)->Aracaju (SE)->Salvador (BH)->Florianopolis (SC)->Rio de Janeiro (RJ)->Campo Grande (MS)->Florianopolis (SC)->Recife (PE)->Campo Grande (MS)->

---
## Exam vB, PROBLEM 6
Maximum Points = 8


## Black box testing

In the following problem we will continue with our SMS spam / nospam data. This time we will try to approach the problem as a pattern recognition problem. For this particular problem I have provided you with everything -- data is prepared, split into train-test sets and a black-box model has been fitted on the training data and predicted on the test data. Your goal is to calculate test metrics and provide guarantees for each metric.

1. [2p] Compute precision for class 1 (see notes 8.3.2 for definition), then provide an interval using Hoeffding's inequality for a 95\% confidence.
2. [2p] Compute recall for class 1(see notes 8.3.2 for definition), then provide an interval using Hoeffding's inequality for a 95\% interval.
3. [2p] Compute accuracy (0-1 loss), then provide an interval using Hoeffding's inequality for a 95\% interval.
4. [2p] If we would have used a classifier with VC-dimension 3, would we have obtained a smaller interval for accuracy by using all data?

In [55]:

# The code below will load data, split the data into train and test and run a "black box" algorithm on it
# the result of the "black box" is stored in predictions_problem6, the true values will be stored in
# Y_test_problem6
from Utils import load_sms
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

data = load_sms()
df = pd.DataFrame(data, columns=["text", "spam"])
print(df.info())

X_train_problem6,X_test_problem6,Y_train_problem6,Y_test_problem6 = train_test_split(df["text"],df["spam"], random_state=42, test_size=0.2)

clf = Pipeline(
    [('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier(n_neighbors=4)),
])

predictions_problem6 = clf.fit(X_train_problem6, Y_train_problem6).predict(X_test_problem6)
print(predictions_problem6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5572 non-null   object
 1   spam    5572 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 87.2+ KB
None
[0 0 0 ... 0 0 0]


In [56]:

# Compute the precision of predictions_problem6 with respect to Y_test_problem6
cond = sum([1 if p==1 and y==1 else 0 for p, y in zip(predictions_problem6, Y_test_problem6)])
problem6_precision = cond/sum(predictions_problem6)
print(problem6_precision)

1.0


In [67]:

# Compute the interval length l of precision of predictions_problem6 with respect to Y_test_problem6, with the same definition of l as in problem 4
problem6_precision_l = np.sqrt((-1/(2*sum(predictions_problem6)))*np.log(0.05/2))
print(problem6_precision_l)

0.18148394886999197


In [59]:

# Repeat the same procedure but for recall
cond = sum([1 if p==1 and y==1 else 0 for p, y in zip(predictions_problem6, Y_test_problem6)])
problem6_recall = cond/sum(Y_test_problem6)
print(problem6_recall)

0.37333333333333335


In [68]:

problem6_recall_l = np.sqrt((-1/(2*sum(Y_test_problem6)))*np.log(0.05/2))
print(problem6_recall_l)

0.11088852441549782


In [65]:

# Repeat the same procedure but for accuracy or 0-1 loss
print(clf.score(X_test_problem6, Y_test_problem6))
problem6_accuracy = sum([1 if p==y else 0 for p, y in zip(predictions_problem6, Y_test_problem6)])/len(Y_test_problem6)
print(problem6_accuracy)

0.915695067264574
0.915695067264574


In [69]:

problem6_accuracy_l = np.sqrt((-1/(2*len(Y_test_problem6)))*np.log(0.05/2))
print(problem6_accuracy_l)

0.04067193176178074


In [71]:

# Below you will calculate the interval parameter l for a classifier running on all data with a VC dimension of 3
# put the value in problem6_VC_l and answer problem_VC_smaller as True if the interval is smaller than the test-accuracy above
# if not answer False. Make sure you replace XXX with something even if you only answer one of them.
problem6_VC_l = np.sqrt(-(64*np.log(0.95/(8*(len(data)*np.e/3)**3)))/len(data)) # number
print(problem6_VC_l)
problem6_VC_smaller = False

0.5641750424582807
